In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt

plt.rcParams['figure.figsize'] = [10, 10]
plt.rcParams['image.origin'] = 'lower'
#plt.rcParams.update({'font.size': 17})

SMALL_SIZE = 15
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

#plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

def savefig(filename):
    plt.savefig("plots/"+filename, dpi=70, bbox_inches = 'tight', pad_inches = 0.1)

In [ ]:
from time import time 

# Load Time

In [ ]:
%%time
t_load_start = time()

from astropy.io import fits 

image = fits.getdata('./bg_subtracted_abell2744_test_image.fits')
header = fits.getheader('./bg_subtracted_abell2744_test_image.fits')
rms =  fits.getdata('./bg_subtracted_abell2744_test_rms.fits')

# Load PSF image (2D array)
PSF = fits.getdata('f105w_psf.fits.gz')

# Normalize PSF 
PSF = PSF / PSF.sum()

t_load_end = time()

In [ ]:
load_time = t_load_end - t_load_start # Seconds
load_time

In [ ]:
# Note that the PSF shape is odd on all sides
print("PSF Shape = {}".format(PSF.shape))

# Plot PSF and use vmax and vmin to show difraction spikes
plt.imshow(PSF, vmin=0, vmax=PSF.std()/10)
plt.show()

In [ ]:
import numpy as np


vmax = 0.05 # Use the image std as max and min of all plots 
vmin = - vmax 

plt.imshow(image, vmin=vmin, vmax=vmax)
plt.title("Galaxy in Abell 2744")
plt.xlabel("Pixels")
plt.ylabel("Pixels")
plt.show()

## Make Source Catalog 

We will use the sigma clipped std as a threshold at the segmentation and deblending steps.


In [ ]:
from astropy.stats import sigma_clipped_stats
image_mean, image_median, image_stddev = sigma_clipped_stats(image.data, sigma=3)

Here we identity sources in the input image.

In [ ]:
from petrofit.segmentation import make_catalog, plot_segments

threshold = image_stddev * 2

# Define smoothing kernel
kernel_size = 3
fwhm = 3

# Min Source size (area)
npixels = 2**2


cat, segm, segm_deblend = make_catalog(    
    image, 
    threshold=threshold, 
    deblend=True,                 
    kernel_size=kernel_size,                  
    fwhm=fwhm, 
    npixels=npixels,
    contrast=0.00,
    plot=True, vmax=vmax, vmin=vmin
)

plt.show()

# Display source properties
print("Num of Targets:", len(cat))

In [ ]:
from petrofit.photometry import order_cat
from petrofit.utils import plot_target
# Sort and select object of interest in the catalog
sorted_idx_list = order_cat(cat, key='area', reverse=True)
idx = sorted_idx_list[2] # index 0 is largest 
source = cat[idx]  # get source from the catalog 

plot_target(
    position=(source.maxval_xindex, source.maxval_yindex), 
    image=image, 
    size=max(image.data.shape)//2, 
    vmax=vmax, 
    vmin=vmin
)

In [ ]:
source.label

# PetroFit

# Run Time

In [ ]:
run_time_start = time()


from petrofit.photometry import make_radius_list
from petrofit.photometry import source_photometry

max_pix = 70

r_list = make_radius_list(
    max_pix=max_pix, # Max pixel to go up to
    n=max_pix # the number of radii to produce 
)

print("len(r_list) = {}".format(len(r_list)))


# Photomerty 
flux_arr, area_arr, error_arr = source_photometry(
    
    # Inputs 
    source, # Source (`photutils.segmentation.catalog.SourceCatalog`)
    image, # Image as 2D array 
    segm_deblend, # Deblended segmentation map of image
    r_list, # list of aperture radii
    
    error=rms,
    
    # Options 
    cutout_size=max(r_list)*2, # Cutout out size, set to double the max radius  
    bkg_sub=True, # Subtract background  
    sigma=3, sigma_type='clip', # Fit a 2D plane to pixels within 1 sigma of the mean
    plot=False, vmax=vmax, vmin=vmin, # Show plot with max and min defined above
)

from petrofit.petrosian import Petrosian, PetrosianCorrection

p = Petrosian(r_list, area_arr, flux_arr)

pc = PetrosianCorrection("../example_correction_gid.yaml")

corrected_epsilon = pc.estimate_epsilon(
    p.r_half_light,
    p.concentration_index()[-1]

)

corrected_p = Petrosian(r_list, area_arr, flux_arr, uncertainties=error_arr,
                        epsilon=corrected_epsilon)


from petrofit import segmentation 

image_x_0, image_y_0 = segmentation.get_source_position(source)
ellip = segmentation.get_source_ellip(source)
elong = segmentation.get_source_elong(source)
theta = segmentation.get_source_theta(source)
n = pc.estimate_n(
    p.r_half_light,
    p.concentration_index()[-1]
)
r_eff = corrected_p.r_half_light

from photutils.isophote import EllipseGeometry, Ellipse

# Define EllipseGeometry using ellip and theta
g = EllipseGeometry(image_x_0, image_y_0, 1., ellip, theta)

# Create Ellipse model 
ellipse = Ellipse(image, geometry=g)

# Fit isophote at r_eff
iso = ellipse.fit_isophote(r_eff)

# Get flux at r_eff
amplitude = iso.intens


from astropy.nddata import Cutout2D
from petrofit.segmentation import masked_segm_image

# Define cutout size 
cutout_size = 150 #max(r_list)*2

# Make an image with all sources masked except `source`
masked_image = masked_segm_image(source, image, segm_deblend, fill=None, mask_background=False)

# Make cutouts 
fitting_image = Cutout2D(masked_image, (image_x_0, image_y_0), cutout_size, mode='partial', copy=True)
fitting_image_unmasked = Cutout2D(image.data, (image_x_0, image_y_0), cutout_size, mode='partial', copy=True)

# Define new center 
x_0 = y_0 = cutout_size / 2

from astropy.modeling import models 

center_slack = 4

sersic_model = models.Sersic2D(
    
        amplitude=amplitude,
        r_eff=r_eff,
        n=n,
        x_0=x_0,
        y_0=y_0,
        ellip=ellip, 
        theta=theta,
    
        bounds = {
            'amplitude': (0., None),
            'r_eff': (0, None),
            'n': (0, 10),
            'ellip': (0, 1),
            'theta': (-2*np.pi, 2*np.pi),
            'x_0': (x_0 - center_slack/2, x_0 + center_slack/2),
            'y_0': (y_0 - center_slack/2, y_0 + center_slack/2),
        },
)

from petrofit.models import PSFModel

oversample = ('x_0', 'y_0', 20, 5)
psf_sersic_model = PSFModel.wrap(sersic_model, psf=PSF, oversample=oversample)

# Limit PSF rotation to -5 to 5 degrees
psf_sersic_model.bounds['psf_pa'] = (-5, 5)

# To disable the PSF rotation, 
# you can set the psf_pa to fixed.
psf_sersic_model.fixed['psf_pa'] = True

from petrofit.fitting import fit_model
fitted_model, fit_info = fit_model(
    fitting_image.data, psf_sersic_model,
    maxiter=10000,
    epsilon=1.4901161193847656e-08,
    acc=1e-09,
)
run_time_end = time()
run_time_end

In [ ]:
run_time = run_time_end - run_time_start
run_time

In [ ]:
total_time = load_time + run_time 
total_time

In [ ]:
from astropy.table import Table
rows = ['amplitude', 'r_eff', 'n', 'x_0', 'y_0', 'ellip', 'theta', 'r_petro', 'r_20', 'r_80', 'C2080', 'Total Flux']
rows

In [ ]:
from petrofit.fitting import print_model_params

print_model_params(fitted_model)
print('{:0.3f} r_petro'.format(corrected_p.r_petrosian))
print('{:0.3f} r_20'.format(corrected_p.fraction_flux_to_r(0.2)))
print('{:0.3f} r_80'.format(corrected_p.fraction_flux_to_r(0.8)))
print('{:0.3f} C2080'.format(corrected_p.c2080))
print('{:0.3f} Total Flux'.format(corrected_p.total_flux))

In [ ]:
from petrofit.photometry import flux_to_abmag
flux_to_abmag(float(corrected_p.total_flux), header)